# Chapter 3 - 深入看看 Transformer LLM (Looking Inside Transformer LLMs)

## 3.1  加载 model 和 tokenizer

In [2]:
# !pip install transformers>=4.41.2 # accelerate>=0.31.0

In [1]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [1]:
# !pip install accelerate

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = r"/openbayes/home/chapter3/Qwen2.5-0.5B-Instruct"
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

# Create a pipeline
generator = pipeline( 
    "text-generation", # 创建一个用于 文本生成 的管道（pipeline）
    model=model, # 指定你要使用的预训练模型
    tokenizer=tokenizer, # 指定与模型配套的分词器
    return_full_text=False, # 控制是否返回完整的文本 / 如果设为 False，只返回模型生成的新内容 / 如果设为 True，返回的是 input + generated text
    max_new_tokens=50,  # 限制最多生成 50 个新 token （一个词可能对应多个 token）
    do_sample=False, # 控制是否使用 采样策略（如 temperature、top-k、top-p）如果设为 False，使用 贪婪解码（每一步都选概率最高的 token）/ 如果设为 True，则使用采样策略，生成更有创意但可能不稳定的文本
)

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [4]:
prompt = "春风又绿江南岸 是谁写的？"
output = generator(prompt)

print(output[0]['generated_text'])

（ ） A. 李白 B. 白居易 C. 苏轼 D. 王安石
李白

“春眠不觉晓，处处闻啼鸟。夜来风雨声，花落知多少


In [5]:
output

[{'generated_text': '（ ） A. 李白 B. 白居易 C. 苏轼 D. 王安石\n李白\n\n“春眠不觉晓，处处闻啼鸟。夜来风雨声，花落知多少'}]

### 备注 
text-generation 仅仅是预测下一个token，所以相对于 如果没有构造成 [chat] 类的格式，效果会更差一些

为什么这个output 后面还有一段跟问题不相干的？ 

- 这是因为你使用的是一个 文本生成模型，它的行为是：根据输入内容继续生成可能合理的文本，而不是只回答问题或填空。

- 模型看到这个格式后，会尝试“补全”这个文本，猜测你是在出题或写文章

- 这是一首 白居易的《春晓》，但模型错误地归到了“李白”名下——这也说明它并不是在做逻辑推理，而是在做语言补全

🤖 模型的生成逻辑是：

- 看到选择题格式 → 猜测你想要一个答案 → 生成“李白”

- 看到“李白” → 联想到古诗 → 继续生成一首相关诗句

但它并不会验证诗句是否真的出自李白

In [ ]:
print(model)

## 3.2 面试要点- RMSNorm 和 layernorm 的区别？

RMSNorm 与 LayerNorm 的主要区别在于是否进行中心化处理：

- LayerNorm 使用均值和标准差归一化
- RMSNorm 只使用均方根（RMS），省略了均值计算，从而更高效。
- RMSNorm 可学习参数少于 layernorm,计算量更小。
- RMSNorm 只有缩放操作，没有 recenter 的操作。

RMSNorm 和 LayerNorm 都是 **归一化方法**，用于深度学习模型中提升训练稳定性和收敛速度，尤其在 Transformer 架构中非常常见。它们的目标是：**对每个样本的特征进行标准化处理，使模型更容易学习。**

### 🧪 LayerNorm（Layer Normalization）

**原理：**
- 对每个样本的所有特征维度进行归一化
- 减去均值，再除以标准差
- 公式：
  $\text{LayerNorm}(x) = \gamma \cdot \frac{x - \mu}{\sigma + \epsilon} + \beta$
  $, \sigma = \sqrt{ \frac{1}{n} \sum (x - \mu)^2 }$
  - $\mu$：均值
  - $\sigma$：标准差
  - $\gamma, \beta$：可学习的缩放和偏移参数

**特点：**
- 保证输出均值为 0，方差为 1
- 在 BERT、GPT 等模型中广泛使用

### ⚡ RMSNorm（Root Mean Square Normalization）

**原理：**
- 不减均值，只除以均方根（RMS）
- 更简单、更快
- 公式：
  $
  \text{RMSNorm}(x) = \gamma \cdot \frac{x}{\sqrt{\frac{1}{n} \sum x_i^2} + \epsilon}
  $
  - 没有中心化（不减均值）
  - 只进行尺度归一化

**特点：**
- 更高效，计算量更小
- 在 LLaMA、RWKV 等新模型中使用
- 更适合深层网络，梯度更稳定


🧠 应用建议

如果你在使用传统 Transformer 模型（如 BERT），LayerNorm 是默认选择。

如果你在构建或微调 LLaMA、RWKV、Mistral 等新架构，RMSNorm 更适合，尤其在资源受限或深层网络中。

## 3.3 查看一个 token 的概率分布（采样和解码）

这段代码的作用是将文本 "The capital of France is" 编码成模型可以理解的张量（tensor），并将其移动到 GPU 上以加速后续的推理或训练。

In [13]:
prompt = "The capital of France is"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
# .input_ids 提取出编码后的 token ID 张量。

input_ids = input_ids.to("cuda")

In [14]:
input_ids

tensor([[ 785, 6722,  315, 9625,  374]], device='cuda:0')

例如，"The capital of France is" 被分词器编码成了：
```
tensor([[ 785, 6722,  315, 9625,  374]])
```
这些数字是每个词或子词的 token ID，它们是模型词表中的索引。

In [15]:
# Get the output of the model before the lm_head
model_output = model.model(input_ids)
model_output

BaseModelOutputWithPast(last_hidden_state=tensor([[[ -5.5625,  -7.4062,   6.5000,  ...,  -2.1719,  -7.2812,   8.8125],
         [ -6.6875,  -4.5000,  -0.3691,  ...,   9.6875,  -1.8203,  -4.8438],
         [ -6.5625,   9.7500,   2.9375,  ...,   4.7812,   8.1250,   0.1108],
         [ -2.0312,   1.3594,   5.3438,  ...,  -0.4570,  -5.4062, -21.5000],
         [ -2.9844,  -8.1250,   4.3750,  ..., -11.0000,   0.9297,  -5.4062]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<MulBackward0>), past_key_values=DynamicCache(layers=[DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer]), hidden_states=None, attentions=None)

这段输出是 Hugging Face Transformer 模型在推理时返回的结构化结果，类型是 BaseModelOutputWithPast。它包含了模型对输入 input_ids 的处理结果。返回的是一个包含多个字段的对象，主要包括：

- last_hidden_state

- past_key_values

- hidden_states

- attentions


last_hidden_state
```
last_hidden_state=tensor([[[...]]], device='cuda:0', dtype=torch.bfloat16)
```
- 是一个三维张量，形状为 (batch_size, sequence_length, hidden_size)。

- 每个 token（如 "The", "capital", "of", ...）都会被编码成一个向量（embedding），这些向量就是 hidden states。

- 这些向量是模型对每个 token 的语义表示，后续可以用于生成、分类、相似度计算等任务。

past_key_values
```
past_key_values=DynamicCache(layers=[DynamicLayer, DynamicLayer, ...])
```
- 这是 Transformer 解码器的缓存机制，用于加速自回归生成。

- 每个 DynamicLayer 对应一个 Transformer 层的注意力缓存（key 和 value）。

- 在生成任务中，使用 past_key_values 可以避免重复计算历史 token 的注意力，从而加速生成。

hidden_states=None：如果你在模型配置中设置了 output_hidden_states=True，这里会返回所有层的 hidden states。

attentions=None：如果设置了 output_attentions=True，这里会返回注意力权重矩阵。

In [16]:
# Get the output of the lm_head
lm_head_output = model.lm_head(model_output[0])
lm_head_output

tensor([[[ 2.1875,  0.1992, -1.1172,  ..., -2.0625, -2.0625, -2.0625],
         [ 5.7188,  5.0000,  1.0703,  ..., -1.9062, -1.9062, -1.9062],
         [ 2.6719, -0.5703, -2.7656,  ..., -5.0625, -5.0625, -5.0625],
         [10.0000,  6.6562,  3.7031,  ..., -3.7812, -3.7812, -3.7812],
         [ 7.5938,  4.2188,  3.9531,  ..., -4.6250, -4.6250, -4.6250]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>)

lm_head 是语言模型的输出层，通常是一个线性层（nn.Linear），它的作用是将隐藏状态（维度为 hidden_size）映射到词表大小（vocab_size）。

换句话说，它把每个 token 的向量变成一个对所有词的打分（logits）。

In [20]:
# 896 是模型的 config.hidden_state
lm_head_output.shape

torch.Size([1, 5, 151936])

📊 输出 lm_head_output 是什么？

- 是一个三维张量，形状为 (batch_size, sequence_length, vocab_size)。

- 每个位置的向量表示该 token 之后可能出现的下一个词的打分（logits）。

- 你可以对这个输出做 argmax 得到预测的下一个 token，或者做 softmax 得到概率分布。




| 维度位置 | 含义                             | 示例值   |
| -------- | -------------------------------- | -------- |
| `[0]`    | batch size（这里是 1）           | `1`      |
| `[1]`    | 输入序列长度（token 数）         | `5`      |
| `[2]`    | 词表大小（每个 token 的 logits） | `151936` |

所以这个张量表示：对每个输入 token，模型都输出了一个长度为 151936 的向量，表示对词表中每个词的预测打分（logits）。

In [22]:
model_output[0].shape

torch.Size([1, 5, 896])

🔍 `model_output[0].shape == torch.Size([1, 5, 896])` 的含义

| 维度位置 | 含义                                 | 示例值 |
| -------- | ------------------------------------ | ------ |
| `[0]`    | batch size（这里是 1）               | `1`    |
| `[1]`    | 输入序列长度（token 数）             | `5`    |
| `[2]`    | hidden size（每个 token 的向量维度） | `896`  |

所以这个张量表示：模型对每个输入 token（比如 `"The"`, `"capital"`, `"of"` 等）都输出了一个长度为 896 的向量，这些向量就是 token 的语义表示。

In [26]:
model.config.hidden_size

896

In [27]:
model.config

Qwen2Config {
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "dtype": "bfloat16",
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention"
  ],
  "max_position_embeddings": 32768,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_

假设你输入的是 "The capital of France is"，模型的最后一个 hidden state 是 model_output.last_hidden_state[:, -1, :]，你可以这样预测下一个词：

In [17]:
token_id = lm_head_output[0, -1].argmax(-1)
token_id

tensor(12095, device='cuda:0')

In [18]:
tokenizer.decode(token_id) # 得到预测结果

' Paris'

### 备注 
这里有两个 API， model.model() 和 model.lm_head()
> model.model 是获取每一个 token hidden state, lm_head 是做 softmax 判断每一词是什么？ 

## 3.4 使用 KV cache 加速

KV cache（Key-Value 缓存）是 Transformer 解码器中的一种加速机制。它的作用是：

- 在生成文本时，模型每次只需要处理新输入的 token，而不是重新计算整个序列。。

- 它会缓存之前每一层的注意力 key 和 value，从而避免重复计算。

In [23]:
prompt = "Write a very long email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

In [24]:
%%timeit -n 1
# Generate the text
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=1000,
  use_cache=True
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


16 s ± 3.53 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


使用 KV cache 来加速生成。

输出提示：你没有设置 attention_mask，所以它默认忽略 padding。

时间结果：约 16 秒。

In [25]:
%%timeit -n 1
# Generate the text
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=1000,
  use_cache=False
)

16 s ± 5.2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


不使用缓存，每次都重新计算整个序列。

时间结果：也是约 16 秒。